In [ ]:
%cd /content

import IPython.display
from IPython.core.display import display, HTML
import os
import sys
import random
import string
import urllib.parse
import subprocess
import threading
import time
import re
import socket

if not os.path.exists("/content/sd-inference-server"):
    !git clone https://github.com/arenasys/sd-inference-server.git
    %cd /content/sd-inference-server
    !git clone https://github.com/arenasys/sd-scripts.git training

%cd /content/sd-inference-server
!git pull

model_folder = "/content/sd-inference-server/models"
try:
    # decline the popup to use the local folder ^
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    model_folder = "/content/drive/My Drive/qDiffusion/models"
    if not os.path.exists(model_folder):
        !mkdir '/content/drive/My Drive/qDiffusion' -p
        !cp -r 'models/' '/content/drive/My Drive/qDiffusion/'
except Exception as e:
    pass

if not os.path.exists("venv"):
    !mkdir venv
    !pip uninstall -q -y tensorflow
    !pip install -q diffusers==0.21.0 k_diffusion==0.0.15 spandrel==0.1.8 lark==1.1.5 transformers==4.36.2 websockets==11.0.3 bson==0.5.10 mega.py==1.0.8 pytorch-lightning==2.0.2 timm==0.9.2 tomesd==0.1.3 pycloudflared==0.2.0 segment-anything==1.0 geffnet==1.0.2 toml==0.10.2 voluptuous==0.13.1 accelerate==0.27.2
    IPython.display.clear_output()

    !apt -y update -qq
    !wget https://github.com/camenduru/gperftools/releases/download/v1.0/libtcmalloc_minimal.so.4 -O /content/libtcmalloc_minimal.so.4
    %env LD_PRELOAD=/content/libtcmalloc_minimal.so.4
    IPython.display.clear_output()

    !wget https://github.com/cloudflare/cloudflared/releases/download/2024.3.0/cloudflared-linux-amd64 -O cloudflared
    !chmod +x cloudflared
    IPython.display.clear_output()

if not sys.path[0] == "/content/sd-inference-server/":
    sys.path.insert(0, "/content/sd-inference-server/")

IPython.display.clear_output()
print("STARTING...")
password = ''.join(random.SystemRandom().choice(string.ascii_letters + string.digits) for _ in range(8))
stopping = False

def tunnel_thread():
    while True:
        time.sleep(0.1)
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        result = sock.connect_ex(('127.0.0.1', 28888))
        if result == 0:
            break
        sock.close()
    time.sleep(1)

    url_pattern = re.compile(r"(?P<url>https?://\S+\.trycloudflare\.com)")
    p = subprocess.Popen(["./cloudflared", "tunnel", "--url", "http://127.0.0.1:28888"], stdout=subprocess.DEVNULL, stderr=subprocess.PIPE, encoding="utf-8")
    for line in p.stderr:
        url_match = url_pattern.search(line)
        if url_match:
            tunnel_url = url_match.group("url")
            endpoint = tunnel_url.replace("https", "wss").strip()
            params = urllib.parse.urlencode({'endpoint': endpoint, "password": password})
            print("ENDPOINT:", endpoint)
            display(HTML(f'<pre>DESKTOP: <a href="{"qdiffusion:?" + params}">{"qdiffusion:?" + params}</a></pre>'))
            print("WEB:", "https://arenasys.github.io/?" + params)
            print("READY!")
            break
    while not stopping:
        time.sleep(0.1)
    print("DIE")
    p.kill()

print("PASSWORD:", password)
threading.Thread(target=tunnel_thread, daemon=True).start()
!python remote.py --models "$model_folder" --password "$password"
stopping = True